<a href="https://colab.research.google.com/github/caioitalo/soulcode/blob/main/Projeto_Aquecimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXTRAÇÃO DOS DADOS


## Importação das bibliotecas e do conector do mongo

In [320]:
import pandas as pd
import numpy as np
import pymongo
from pymongo import MongoClient

In [321]:

uri = "mongodb+srv://caio-soulcode.o09ab0z.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,
                     tls=True,
                     tlsCertificateKeyFile='/content/X509-cert-1702224064282282452.pem')

db = client['proj_aquec']
colecao1 = db['202201_orig']
colecao2 = db['202204_orig']
colecao3 = db['202210_orig']
doc_count = colecao1.count_documents({})
print(doc_count)


152140


In [322]:
df1orig = pd.read_csv('/content/D.SDA.PDA.005.CAT.202201orig.csv',sep=';',encoding='ISO-8859-1')
df2orig = pd.read_csv('/content/D.SDA.PDA.005.CAT.202204orig.csv',sep=';',encoding='ISO-8859-1')
df3orig = pd.read_csv('/content/D.SDA.PDA.005.CAT.202210orig.csv',sep=';',encoding='ISO-8859-1')

In [323]:
df1orig_dict = df1orig.to_dict('records')
df2orig_dict = df2orig.to_dict('records')
df3orig_dict = df3orig.to_dict('records')


In [ ]:
df2orig.dtypes

Agente  Causador  Acidente     object
Data Acidente                  object
CBO                            object
CID-10                         object
CNAE2.0 Empregador              int64
CNAE2.0 Empregador.1           object
Emitente CAT                   object
Espécie do benefício           object
Filiação Segurado              object
Indica Óbito Acidente          object
Munic Empr                     object
Natureza da Lesão              object
Origem de Cadastramento CAT    object
Parte Corpo Atingida           object
Sexo                           object
Tipo do Acidente               object
UF  Munic.  Acidente           object
UF Munic. Empregador           object
Data Acidente.1                object
Data Despacho Benefício        object
Data Acidente.2                object
Data Nascimento                object
Data Emissão CAT               object
CNPJ/CEI Empregador            object
dtype: object

In [ ]:
# Houve um erro no momento de inserir os dados no mongoDB, foi checada cada coluna de cada DF para achar alguma inconsistência impeditiva 
# Incons.: Indica Óbito Acidente ('{ñ') - a falta do fechamento das chaves pode estar gerando problema na geração dos documentos no mongoDB
# Origem de Cadastramento CAT ('{ñ class') - mesma justif.

sorted(df3orig['Data Nascimento'].unique())

In [ ]:
# foi dado um replace em todos os dfs com tais inconsistências
#df1orig.replace(['{','}'],'', regex=True, inplace=True)
df2orig.replace(['{','}'],'', regex=True, inplace=True)
#df3orig.replace(['{','}'],'', regex=True, inplace=True)

In [ ]:
# Carregamento para o MongoDB para ser posto em disponibilidade
#colecao1.insert_many(df1orig_dict)
colecao2.insert_many(df2orig_dict)
#colecao3.insert_many(df3orig_dict)

colecao1.count_documents({})

BulkWriteError: ignored

## Enviando o arquivo para o Cloud Storage

In [324]:
#importação das bibliotecas da cloud storage
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [325]:
# Realizando a configuração da conta utilizando a chave json
from google.cloud import storage
import os

serviceAccount = '/content/bold-origin-366512-48dc9e027646.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount


In [ ]:
# Acessando o bucket, e inserindo os arquivos
cliente = storage.Client()
bucket = cliente.get_bucket('projeto-aquecimento15-12')
blob = bucket.blob('D.SDA.PDA.005.CAT.202201orig.csv')
blob.upload_from_filename('/content/D.SDA.PDA.005.CAT.202201orig.csv')

In [ ]:
cliente = storage.Client()
bucket = cliente.get_bucket('projeto-aquecimento15-12')
blob = bucket.blob('D.SDA.PDA.005.CAT.202204orig.csv')
blob.upload_from_filename('/content/D.SDA.PDA.005.CAT.202204orig.csv')

In [ ]:
cliente = storage.Client()
bucket = cliente.get_bucket('projeto-aquecimento15-12')
blob = bucket.blob('D.SDA.PDA.005.CAT.202210orig.csv')
blob.upload_from_filename('/content/D.SDA.PDA.005.CAT.202210orig.csv')

# CARREGAMENTO INICIAL DOS DADOS

Agora com os arquivos com disponibilidade em nuvem (GCP e MongoDB), serão carregados para transformação

In [376]:
# Carregando os DBs e os transformando em DFs do Pandas
df10 = pd.read_csv('https://storage.googleapis.com/datalake-soulcode-ed7/arquivos%20soulcode/D.SDA.PDA.005.CAT.202201orig.csv',sep=';',encoding = 'ISO-8859-1')
df20 = pd.read_csv('https://storage.googleapis.com/datalake-soulcode-ed7/arquivos%20soulcode/D.SDA.PDA.005.CAT.202204orig.csv',sep=';',encoding = 'ISO-8859-1')
df30 = pd.read_csv('https://storage.googleapis.com/datalake-soulcode-ed7/arquivos%20soulcode/D.SDA.PDA.005.CAT.202210orig.csv',sep=';',encoding = 'ISO-8859-1')

In [377]:
# unindo as três tabelas, usando o concat pois têm o mesmo número de colunas e o que muda são as datas
df = pd.concat([df10,df20,df30])

In [333]:
# gerando um backup
dfback = df.copy()

#TRANSFORMAÇÃO COM PANDAS

## Análises primárias do DF


In [378]:
df

,Agente Causador Acidente,Data Acidente,CBO,CID-10,CNAE2.0 Empregador,CNAE2.0 Empregador.1,Emitente CAT,Espécie do benefício,Filiação Segurado,Indica Óbito Acidente,...,Sexo,Tipo do Acidente,UF Munic. Acidente,UF Munic. Empregador,Data Acidente.1,Data Despacho Benefício,Data Acidente.2,Data Nascimento,Data Emissão CAT,CNPJ/CEI Empregador
0,{ñ class},2022/01,515105-Agente Comunitário de Saúde,B34.2 Infecc p/Coronavirus Ne,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,...,Feminino,Doença,Maranhão,São Paulo,2022/01,0000/00,20/01/2022,02/08/1970,01/03/2022,57.571.275.002.570
1,"Motocicleta, Motoneta",2022/02,519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,...,Masculino,Trajeto,Maranhão,São Paulo,2022/02,0000/00,25/02/2022,16/12/1990,01/03/2022,19.578.086.000.162
2,"Motocicleta, Motoneta",2022/02,519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,...,Masculino,Trajeto,Maranhão,São Paulo,2022/02,0000/00,25/02/2022,16/12/1990,01/03/2022,19.578.086.000.162
3,"Aprision. Em, Sobre ou Entre Dois ou Mai",2022/02,514205-Coletor Lixo,S60.0 Contusao de Dedos s/Lesao da Unha,3811,Coleta de Residuos Nao-Perigosos,Empregador,Pa,Empregado,Não,...,Masculino,Típico,{ñ class},Goiás,2022/02,0000/00,24/02/2022,19/09/1986,01/03/2022,00.000.000.000.000
4,Impacto de Pes. Contra Objeto em Movimento,2022/01,{ñ class},S90.0 Contusao do Tornozelo,4639,Comercio Atacadista de Produtos Alimenticios,Empregador,Pa,Empregado,Não,...,Masculino,Típico,Maranhão,São Paulo,2022/01,0000/00,28/01/2022,25/07/1995,01/03/2022,00.000.000.000.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15654,Piso de Edificio - Superficie Utilizada Para,2022/09,{ñ class},S62 Frat ao Nivel do Punho e da Mao,2330,"Fabricacao de Artefatos de Concreto, Cimento,",{ñ class},Auxílio Doenca por Acidente do Trabalho,Empregado,Não,...,Masculino,Típico,Roraima,Paraná,2022/09,2022/10,29/09/2022,18/09/1993,10/10/2022,21.161.901.000.135
15655,Escada Permanente Cujos Degraus Permitem Apoi,2022/10,232130-Prof. Física no Ensino Médio,S82 Frat da Perna Incl Tornozelo,8520,Ensino Medio,Autoridade Pública,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,...,Feminino,Típico,Roraima,Paraná,2022/10,2022/11,06/10/2022,24/11/1986,07/10/2022,76.416.965.000.121
15656,"Andaime, Plataforma - Edificio ou Estrutura",2022/09,517330-Vigilante,S82.7 Frat Mult da Perna,8112,Condominios Prediais,Segurado/Dependente,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,...,Masculino,Típico,{ñ class},Santa Catarina,2022/09,2022/10,20/09/2022,02/03/1986,10/10/2022,32.994.401.000.109
15657,"Martelo, Malho, Marreta- Ferramenta Manual Se",2022/10,724315-Soldador,S62.6 Frat de Outr Dedos,7112,Servicos de Engenharia,Empregador,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,...,Masculino,Típico,Roraima,Paraná,2022/10,2022/10,05/10/2022,08/04/1984,06/10/2022,34.197.591.000.150


In [335]:
df.dtypes

Agente  Causador  Acidente     object
Data Acidente                  object
CBO                            object
CID-10                         object
CNAE2.0 Empregador              int64
CNAE2.0 Empregador.1           object
Emitente CAT                   object
Espécie do benefício           object
Filiação Segurado              object
Indica Óbito Acidente          object
Munic Empr                     object
Natureza da Lesão              object
Origem de Cadastramento CAT    object
Parte Corpo Atingida           object
Sexo                           object
Tipo do Acidente               object
UF  Munic.  Acidente           object
UF Munic. Empregador           object
Data Acidente.1                object
Data Despacho Benefício        object
Data Acidente.2                object
Data Nascimento                object
Data Emissão CAT               object
CNPJ/CEI Empregador            object
dtype: object

In [389]:
df.isna().sum()

ag_causador         0
cbo                 0
cid10               0
cod_cnae            0
desc_cnae           0
emitente_cat        0
tipo_beneficio      0
filiacao_seg        0
obito               0
munic_empregador    0
nat_lesao           0
local_da_lesao      0
sexo                0
tipo_acid           0
uf_acid             0
uf_empregador       0
dt_acid             0
dt_desp_ben         0
data                0
dt_nasc             0
dt_emissao_cat      0
cnpj_cei            0
dtype: int64

In [379]:
# Comparando as duas colunas da tabela
df['Data Acidente.1'].equals(df['Data Acidente'])

True

In [380]:
# dropando uma das colunas iguais
# dropando Origem do cadastramento, pois só exibe um valor, sendo irrelevante para as análises.
df.drop('Data Acidente',axis=1,inplace=True)
df.drop('Origem de Cadastramento CAT',axis=1,inplace=True)

In [381]:
# Renomeando as colunas para deixá-las mais fáceis
df.rename(columns={'Agente  Causador  Acidente':'ag_causador', 'CID-10':'cid10','CNAE2.0 Empregador':'cod_cnae',
                   'CNAE2.0 Empregador.1':'desc_cnae','Emitente CAT':'emitente_cat','Espécie do benefício':'tipo_beneficio',
                   'Filiação Segurado':'filiacao_seg','Indica Óbito Acidente':'obito','Munic Empr':'munic_empregador',
                   'Natureza da Lesão':'nat_lesao','Parte Corpo Atingida':'local_da_lesao','Sexo':'sexo','Tipo do Acidente':'tipo_acid',
                   'UF  Munic.  Acidente':'uf_acid','UF Munic. Empregador':'uf_empregador','Data Acidente.1':'dt_acid',
                    'Data Despacho Benefício':'dt_desp_ben', 'Data Acidente.2':'data','Data Nascimento':'dt_nasc','Data Emissão CAT':'dt_emissao_cat',
                   'CNPJ/CEI Empregador':'cnpj_cei','CBO':'cbo'},inplace=True)

In [360]:
df.dtypes

ag_causador         object
cbo                 object
cid10               object
cod_cnae             int64
desc_cnae           object
emitente_cat        object
tipo_beneficio      object
filiacao_seg        object
obito               object
munic_empregador    object
nat_lesao           object
local_da_lesao      object
sexo                object
tipo_acid           object
uf_acid             object
uf_empregador       object
dt_acid             object
dt_desp_ben         object
data                object
dt_nasc             object
dt_emissao_cat      object
cnpj_cei            object
dtype: object

In [414]:
# usando unique novamente para checar inconsistências de cada coluna
# Inconsistências: valores com muitos espaços - ok, {ñ class} ,reverse=True
# cod_cnae: olhar o '0'
# tipo_beneficio: 'Pa' é uma inconsistência?
# 

df['filiacao_seg'].unique()

array(['Empregado', 'Trabalhador Avulso', nan, 'Segurado Especial'],
      dtype=object)

In [385]:
# Usando python para percorrer as colunas e retirar o excesso de espaços. foi necessário transformar cod_cnae para utilizar esse método.
df['cod_cnae'] = df['cod_cnae'].astype(str)
for coluna in df:
  df[coluna] = df[coluna].str.strip()


In [396]:
# Trocando os valores inconsistentes 
df.replace('{ñ class}',np.nan,inplace=True)

#TRANSFORMAÇÃO COM PYSPARK

# CARREGAMENTO FINAL